In [1]:
# 셀레니움
from selenium import webdriver 
# 셀레니움 By 클래스 가져오기
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# NoSuchElementException import 추가
from selenium.common.exceptions import NoSuchElementException  
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException
import logging
from selenium.common.exceptions import TimeoutException

# 창 열기

In [2]:
# 크롬 웹 드라이버 초기화 및 카카오 맵 페이지로 이동
driver = webdriver.Chrome() 
url = 'https://m.map.kakao.com/'
driver.get(url)

# 검색창 element와 검색창 id 가져오기
search_box = driver.find_element(By.ID,'innerQuery')
# 키워드 입력
search_box.send_keys("종로구 빵")

# Enter 키 누르기
search_box.send_keys(Keys.RETURN)

# 페이지가 로드될 때까지 잠시 대기
time.sleep(3)

# 베너 닫기
driver.find_element(By.XPATH, f'//*[@id="daumWrap"]/div[1]/div/a[2]/span').click()

# 더보기 버튼을 누를 수 있는지 여부 확인하는 함수
def can_click_load_more_button():
    try:
        load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
        return load_more_button.is_enabled() and load_more_button.is_displayed()
    except:
        return False

# 더보기 버튼 클릭 함수
def click_load_more_button():
    load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
    load_more_button.click()
    time.sleep(3)  # 버튼을 클릭한 후에 충분한 시간 대기 (페이지가 로딩될 때까지)

# 더보기 버튼을 누를 수 없을 때까지 반복해서 클릭
while can_click_load_more_button():
    click_load_more_button()
    
# 페이지에 표시된 정보의 갯수 확인
all_results = driver.find_elements(By.XPATH, '//*[@id="placeList"]/li')
print("페이지에 표시된 정보의 갯수:", len(all_results))
# 스크롤 올리기
driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")

페이지에 표시된 정보의 갯수: 76


# 가게 이름 누르고 후기 더보기 누르기

1. 가게 이름 누르고 들어간다
2. 가게 이름 크롤링
3. rate_intro 찾아서 있으면, 4번, 없으면, 7번
4. 첫번째 후기 클릭해서 전체 후기 들어가기
5. 전체 후기 페이지의 더보기 끝 까지 누르기 
6. 뒤로가기 2번
7. 없으면, 뒤로가기 1번

In [5]:
# 연습 1번
# 2~5초 랜덤한 대기 시간
wait_time = random.uniform(3, 7)

# 가게이름 클릭 함수
def click_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[1]/strong' 
        try:
            driver.find_element(By.XPATH, xpath).click()
        except NoSuchElementException:
            return None

# 스크롤하여 요소가 보이도록 만드는 함수
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

# 후기 더보기 누르기, 후기 접기는 정지
def class_click():
    prev_class = 'link_more'
    while True:
        element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a')
        element_class = element.get_attribute("class")
        # 이전 클래스와 현재 클래스가 다른지 확인
        if prev_class != element_class:
            print("Class changed. Stopping click.")
            break
        # 클릭하고 잠시 대기
        element.click()
        time.sleep(wait_time)
        # 현재 클래스를 이전 클래스에 저장
        prev_class = element_class
                
# 전체 후기 페이지 ID크롤링 
def find_id():
    z = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{z}]/div[1]/div[1]/div[1]/a/span'
        try:
            id_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            ID.append(id_element.text)
            z += 1
            time.sleep(wait_time)
            # 스크롤을 조정하여 다음 요소가 화면에 나타날 수 있도록 함
            # 수직으로 600px만큼 아래로 스크롤
            if z % 3 == 0:
                driver.execute_script("window.scrollBy(0, 600);")  
                time.sleep(wait_time)
        except NoSuchElementException:
            break
  
# 전체 후기 페이지 날짜 크롤링 
def find_date():
    a = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{a}]/div[1]/div[1]/div[2]/span[6]'
        try:
            date_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            Date.append(date_element.text)
            a += 1
            time.sleep(wait_time)
            if a % 3 == 0:
                driver.execute_script("window.scrollBy(0, 600);")  
                time.sleep(wait_time)
        except NoSuchElementException:
            break

# 전체 후기 페이지 리뷰 크롤링 
def find_rv():
    b = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{b}]/div[1]/div[*]/p/span'
        try:
            rv_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
            if review_text:  # 리뷰 텍스트가 존재하면
                Review_text.append(review_text)
            else:  # 리뷰 텍스트가 없는 경우
                Review_text.append('정보 없음')
            b += 1
            time.sleep(wait_time)
            if b % 3 == 0:
                driver.execute_script("window.scrollBy(0, 600);")  
                time.sleep(wait_time)
        except NoSuchElementException:
            break
        
store = []
ID = []
Date = []
Review_text = []

In [24]:
# 연습 1-1
# 2~5초 랜덤한 대기 시간
wait = random.uniform(9, 15)
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 가게이름 클릭 함수
def click_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        logging.info(f"No such element: {xpath}")
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[1]/strong' 
        try:
            driver.find_element(By.XPATH, xpath).click()
        except NoSuchElementException:
            logging.info(f"No such element: {xpath}")
            return None

# 스크롤하여 요소가 보이도록 만드는 함수
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

# 후기 더보기 누르기, 후기 접기는 정지
def class_click():
    prev_class = 'link_more'
    while True:
        element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a')
        element_class = element.get_attribute("class")
        # 이전 클래스와 현재 클래스가 다른지 확인
        if prev_class != element_class:
            print("Class changed. Stopping click.")
            break
        # 클릭하고 잠시 대기
        element.click()
        time.sleep(wait)
        # 현재 클래스를 이전 클래스에 저장
        prev_class = element_class
                
# 전체 후기 페이지 ID크롤링 
def find_id():
    z = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{z}]/div[1]/div[1]/div[1]/a/span'
        try:
            id_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            ID.append(id_element.text)
            z += 1
            time.sleep(wait_time)
            # 스크롤을 조정하여 다음 요소가 화면에 나타날 수 있도록 함
            # 수직으로 600px만큼 아래로 스크롤
            if z % 3 == 0:
                driver.execute_script("window.scrollBy(0, 600);")  
                time.sleep(wait_time)
        except NoSuchElementException:
            logging.info(f"No such element: {xpath}")
            break
  
# 전체 후기 페이지 날짜 크롤링 
def find_date():
    a = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{a}]/div[1]/div[1]/div[2]/span[6]'
        try:
            date_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            Date.append(date_element.text)
            a += 1
            time.sleep(wait_time)
            if a % 3 == 0:
                driver.execute_script("window.scrollBy(0, 600);")  
                time.sleep(wait_time)
        except NoSuchElementException:
            logging.info(f"No such element: {xpath}")
            break

# 전체 후기 페이지 리뷰 크롤링 
def find_rv():
    b = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{b}]/div[1]/div[*]/p/span'
        try:
            rv_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
            if review_text:  # 리뷰 텍스트가 존재하면
                Review_text.append(review_text)
            else:  # 리뷰 텍스트가 없는 경우
                Review_text.append('정보 없음')
            b += 1
            time.sleep(wait_time)
            if b % 3 == 0:
                driver.execute_script("window.scrollBy(0, 600);")  
                time.sleep(wait_time)
        except NoSuchElementException:
            logging.info(f"No such element: {xpath}")
            break
# # 전체 ID, Date, rv 합친 함수
# def find_total():
#     g = 1
#     reviews = []  # 리뷰 정보를 저장할 리스트
#     while True:
#         xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
#         xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
#         xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'
#         try:
#             id_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
#             date_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
#             rv_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
#             review = {
#                 'id': id_element.text,
#                 'date': date_element.text,
#                 'text': rv_element.text.strip() if rv_element.text.strip() else '정보 없음'}
#             reviews.append(review)
#             g += 1
#             if g % 3 == 0:
#                 driver.execute_script("window.scrollBy(0, 600);")
#         except NoSuchElementException:
#             break
# 전체 ID, Date, rv 합친 함수 - 원본
def find_total():
    g = 1
    while True:
        xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
        xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
        xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'
        try:
            # id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
            # ID.append(id_element.text)
            # time.sleep(wait)
            # date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
            # Date.append(date_element.text)
            # time.sleep(wait)
            # rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
            # time.sleep(wait)
            # review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
            # time.sleep(wait)
            # if review_text:  # 리뷰 텍스트가 존재하면
            #     Review_text.append(review_text)
            # else:  # 리뷰 텍스트가 없는 경우
            #     Review_text.append('정보 없음')
            # try:
            #     rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
            #     time.sleep(wait)
            #     review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
            #     if review_text:  # 리뷰 텍스트가 존재하면
            #         Review_text.append(review_text)
            #     else:  # 리뷰 텍스트가 없는 경우
            #         Review_text.append('정보 없음')
            # except (NoSuchElementException, TimeoutException):
            #     Review_text.append('정보 없음')  # xpath_rv에 해당하는 요소가 없는 경우
            # g += 1
            # time.sleep(wait)
            # driver.execute_script("window.scrollBy(0, 300);")  
            prev_scroll_position = None
            while True:
                # 스크롤을 300px 만큼 아래로 
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(1)
                try:
                    id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
                    ID.append(id_element.text)
                    time.sleep(wait)
                    date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
                    Date.append(date_element.text)
                    time.sleep(wait)
                    rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
                    time.sleep(wait)
                    review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
                    if review_text:  # 리뷰 텍스트가 존재하면
                        Review_text.append(review_text)
                    else:  # 리뷰 텍스트가 없는 경우
                        Review_text.append('정보 없음')
                except (NoSuchElementException, TimeoutException):
                    Review_text.append('정보 없음')  # xpath_rv에 해당하는 요소가 없는 경우
                g += 1
                # 현재 스크롤 위치
                curr_scroll_position = driver.execute_script("return window.pageYOffset;")
                # 이전 스크롤 위치가 없으면 현재 스크롤 위치를 이전 위치로 지정
                if prev_scroll_position is None:
                    prev_scroll_position = curr_scroll_position
                    continue
                # 이전 스크롤 위치와 현재 스크롤 위치의 차이를 계산
                scroll_diff = curr_scroll_position - prev_scroll_position
                # 스크롤 위치의 변화가 없다면 멈cna
                if scroll_diff == 0:
                    break
                # 현재 스크롤 위치를 이전 위치로 eocp
                prev_scroll_position = curr_scroll_position
            # time.sleep(wait)
        except NoSuchElementException:
            break

In [29]:
# 연습 1-2

# 랜덤한 대기 시간
wait = random.uniform(9, 15)
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 가게이름 클릭 함수
def click_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        logging.info(f"No such element: {xpath}")
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[1]/strong' 
        try:
            driver.find_element(By.XPATH, xpath).click()
        except NoSuchElementException:
            logging.info(f"No such element: {xpath}")
            return None

# 스크롤하여 요소가 보이도록 만드는 함수
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

# 후기 더보기 누르기, 후기 접기는 정지
def class_click():
    prev_class = 'link_more'
    while True:
        element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a')
        element_class = element.get_attribute("class")
        # 이전 클래스와 현재 클래스가 다른지 확인
        if prev_class != element_class:
            print("Class changed. Stopping click.")
            break
        # 클릭하고 잠시 대기
        element.click()
        time.sleep(wait)
        # 현재 클래스를 이전 클래스에 저장
        prev_class = element_class

# 전체 ID, Date, rv 합친 함수
def find_total():
    g = 1
    while True:
        xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
        xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
        xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'
        try:
            prev_scroll_position = None
            while True:
                # 스크롤을 300px 만큼 아래로 
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(1)
                try:
                    id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
                    ID.append(id_element.text)
                    time.sleep(wait)
                    date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
                    Date.append(date_element.text)
                    time.sleep(wait)
                    rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
                    time.sleep(wait)
                    review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
                    if review_text:  # 리뷰 텍스트가 존재하면
                        Review_text.append(review_text)
                    else:  # 리뷰 텍스트가 없는 경우
                        Review_text.append('정보 없음')
                # xpath_rv에 해당하는 요소가 없는 경우
                except (NoSuchElementException, TimeoutException):
                    ID.append('정보 없음')
                    Date.append('정보 없음')
                    Review_text.append('정보 없음')
                g += 1
                # 현재 스크롤 위치
                curr_scroll_position = driver.execute_script("return window.pageYOffset;")
                # 이전 스크롤 위치가 없으면 현재 스크롤 위치를 이전 위치로 지정
                if prev_scroll_position is None:
                    prev_scroll_position = curr_scroll_position
                    continue
                # 이전 스크롤 위치와 현재 스크롤 위치의 차이를 계산
                scroll_diff = curr_scroll_position - prev_scroll_position
                # 스크롤 위치의 변화가 없다면 멈춤
                if scroll_diff == 0:
                    break
                # 현재 스크롤 위치를 이전 위치로 대체
                prev_scroll_position = curr_scroll_position
        except NoSuchElementException:
            break
# find_total 수정       
def find_data():
    g = 1
    while True:
        xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
        xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
        xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'
        try:
            prev_scroll_position = None
            # 스크롤 다운
            driver.execute_script("window.scrollBy(0, 200);")
            time.sleep(1)
            try:
                id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
                ID.append(id_element.text)
                time.sleep(wait)
                date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
                Date.append(date_element.text)
                time.sleep(wait)
                rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
                time.sleep(wait)
                review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
                if review_text:  # 리뷰 텍스트가 존재하면
                    Review_text.append(review_text)
                else:  # 리뷰 텍스트가 없는 경우
                    Review_text.append('정보 없음')
            # xpath_rv에 해당하는 요소가 없는 경우
            except (NoSuchElementException, TimeoutException):
                ID.append('정보 없음')
                Date.append('정보 없음')
                Review_text.append('정보 없음')
            g += 1
            # # 현재 스크롤 위치 확인
            prev_scroll_position = 0
            curr_scroll_position = driver.execute_script("return window.pageYOffset;")
            # 마지막 스크롤 위치 확인 및 업데이트
            if not 'prev_scroll_position' in locals() or prev_scroll_position < curr_scroll_position:
                prev_scroll_position = curr_scroll_position
            else:
                # 스크롤이 더 이상 변화하지 않는다면, 크롤링 종료
                print("크롤링 완료")
                break
            # 현재 스크롤 위치
            # curr_scroll_position = driver.execute_script("return window.pageYOffset;")
            # # 이전 스크롤 위치가 없으면 현재 스크롤 위치를 이전 위치로 지정
            # if prev_scroll_position is None:
            #     prev_scroll_position = curr_scroll_position
            #     continue
            # # 이전 스크롤 위치와 현재 스크롤 위치의 차이를 계산
            # scroll_diff = curr_scroll_position - prev_scroll_position
            # # 스크롤 위치의 변화가 없다면 멈춤
            # if scroll_diff == 0:
            #     break
            # # 현재 스크롤 위치를 이전 위치로 대체
            # prev_scroll_position = curr_scroll_position
        except NoSuchElementException:
            # 특정 요소를 더 이상 찾을 수 없으므로 크롤링 종료
            print("크롤링 종료: 더 이상 찾을 요소가 없습니다.")
            break
        break
        
# 연습 1-2

In [ ]:
# 리뷰 실패
ID = []
Date = []
Review_text = []
g = 1
while True:
    xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
    xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
    xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'
    prev_scroll_position = 0
    curr_scroll_position = driver.execute_script("return window.pageYOffset;")
    id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
    ID.append(id_element.text)
    time.sleep(wait)
    date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
    Date.append(date_element.text)
    time.sleep(wait)
    rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
    time.sleep(wait)
    review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
    if review_text:  # 리뷰 텍스트가 존재하면
        Review_text.append(review_text)
    else:  # 리뷰 텍스트가 없는 경우
        Review_text.append('정보 없음')
    g += 1
    driver.execute_script("window.scrollBy(0, 200);")
    curr_scroll_position = driver.execute_script("return window.pageYOffset;")
    # 이전 스크롤 위치가 없으면 현재 스크롤 위치를 이전 위치로 지정
    if prev_scroll_position is None:
        prev_scroll_position = curr_scroll_position
        continue
    # 이전 스크롤 위치와 현재 스크롤 위치의 차이를 계산
    scroll_diff = curr_scroll_position - prev_scroll_position
    # 스크롤 위치의 변화가 없다면 멈춤
    if scroll_diff == 0:
        break
    # 현재 스크롤 위치를 이전 위치로 대체
    prev_scroll_position = curr_scroll_position

In [4]:
# 연습 1-3 

# 랜덤한 대기 시간
wait = random.uniform(5,10)
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 가게이름 클릭 함수
def click_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        logging.info(f"No such element: {xpath}")
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[1]/strong' 
        try:
            driver.find_element(By.XPATH, xpath).click()
        except NoSuchElementException:
            logging.info(f"No such element: {xpath}")
            return None

# 스크롤하여 요소가 보이도록 만드는 함수
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

# 후기 더보기 누르기, 후기 접기는 정지
def class_click():
    prev_class = 'link_more'
    while True:
        element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a')
        element_class = element.get_attribute("class")
        # 이전 클래스와 현재 클래스가 다른지 확인
        if prev_class != element_class:
            print("Class changed. Stopping click.")
            break
        # 클릭하고 잠시 대기
        element.click()
        time.sleep(wait)
        # 현재 클래스를 이전 클래스에 저장
        prev_class = element_class



# find_total 수정       
def find_data():
    g = 1  # 루프 밖에서 변수 초기화
    max_attempts = 3  # 최대 시도 횟수를 설정하여 무한 루프 방지
    attempts = 0
    while attempts < max_attempts:
        try:
            xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
            xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
            xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'

            id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
            ID.append(id_element.text)
            time.sleep(wait)

            date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
            Date.append(date_element.text)
            time.sleep(wait)

            try:
                # rv_element를 찾는 시도
                rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
                temp_review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
                if temp_review_text:  # 리뷰 텍스트가 존재하면
                    Review_text.append(temp_review_text)
                else:  # 리뷰 텍스트가 없는 경우
                    Review_text.append('정보 없음')
            except (NoSuchElementException, TimeoutException):
                # 해당 XPATH의 element를 찾지 못한 경우, 아무것도 하지 않고 넘어감
                Review_text.append('정보 없음')

            driver.execute_script("window.scrollBy(0, 200);")  # 페이지 스크롤 다운
            g += 1  # 다음 요소로 이동
            attempts = 0  # 성공적으로 요소를 찾으면 시도 횟수를 초기화
        except (TimeoutException, NoSuchElementException):
            attempts += 1  # 요소를 찾지 못하면 시도 횟수 증가

        # 현재 스크롤 위치를 확인하여 이전 위치와 비교하는 로직은 제거함
        # 대신, 최대 시도 횟수에 도달하면 반복을 종료
        if attempts >= max_attempts:
            print("최대 시도 횟수에 도달하였거나 더 이상 크롤링할 요소가 없습니다.")
            break 
        
# 연습 1-3

In [23]:
# 리뷰 완료
ID = []
Date = []
Review_text = []  # 모든 리뷰 텍스트를 저장할 리스트
g = 122  # 루프 밖에서 변수 초기화
max_attempts = 3  # 최대 시도 횟수를 설정하여 무한 루프 방지
attempts = 0

while attempts < max_attempts:
    try:
        xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
        xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
        xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'

        id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
        ID.append(id_element.text)
        time.sleep(wait)
        date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
        Date.append(date_element.text)
        time.sleep(wait)

        try:
            # rv_element를 찾는 시도
            rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
            temp_review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
            if temp_review_text:  # 리뷰 텍스트가 존재하면
                Review_text.append(temp_review_text)
            else:  # 리뷰 텍스트가 없는 경우
                Review_text.append('정보 없음')
        except (NoSuchElementException, TimeoutException):
            # 해당 XPATH의 element를 찾지 못한 경우, 아무것도 하지 않고 넘어감
            Review_text.append('정보 없음')

        driver.execute_script("window.scrollBy(0, 200);")  # 페이지 스크롤 다운
        g += 1  # 다음 요소로 이동
        attempts = 0  # 성공적으로 요소를 찾으면 시도 횟수를 초기화
    except (TimeoutException, NoSuchElementException):
        attempts += 1  # 요소를 찾지 못하면 시도 횟수 증가

    # 현재 스크롤 위치를 확인하여 이전 위치와 비교하는 로직은 제거함
    # 대신, 최대 시도 횟수에 도달하면 반복을 종료
    if attempts >= max_attempts:
        print("최대 시도 횟수에 도달하였거나 더 이상 크롤링할 요소가 없습니다.")
        break


최대 시도 횟수에 도달하였거나 더 이상 크롤링할 요소가 없습니다.


In [24]:
print(len(Review_text))
print(len(ID))
print(len(Date))

103
103
103


In [18]:
prev_scroll_position = None
while True:
    # 스크롤을 300px 만큼 아래로 내립니다.
    driver.execute_script("window.scrollBy(0, 300);")
    time.sleep(1)  # 스크롤 이동을 기다립니다.

    # 현재 스크롤 위치를 가져옵니다.
    curr_scroll_position = driver.execute_script("return window.pageYOffset;")
    
    # 이전 스크롤 위치가 없으면 현재 스크롤 위치를 이전 위치로 지정합니다.
    if prev_scroll_position is None:
        prev_scroll_position = curr_scroll_position
        continue

    # 이전 스크롤 위치와 현재 스크롤 위치의 차이를 계산합니다.
    scroll_diff = curr_scroll_position - prev_scroll_position
    
    # 스크롤 위치의 변화가 없다면 멈춥니다.
    if scroll_diff == 0:
        break
    # 현재 스크롤 위치를 이전 위치로 갱신합니다.
    prev_scroll_position = curr_scroll_position


더 이상 스크롤할 수 없습니다.


In [53]:
# 완성 1번
# 2~5초 랜덤한 대기 시간
wait_time = random.uniform(3, 7)

# 가게이름 클릭 함수
def click_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[1]/strong' 
        try:
            driver.find_element(By.XPATH, xpath).click()
        except NoSuchElementException:
            return None

# 스크롤하여 요소가 보이도록 만드는 함수
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

# 전체 후기 페이지 더보기 버튼 클릭 함수
# def load_more():
#     load_more = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a/span')
#     load_more.click()
#     time.sleep(wait_time)

# # 전체 후기 페이지의 ID 크롤링
# def scrape_ids():
#     ids = []
#     index = 1
#     while True:
#         xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{index}]/div[1]/div[1]/div[1]/a/span'
#         try:
#             # id_element = driver.find_element(By.XPATH, xpath)
#             id_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath)))
#             ids.append(id_element.text)
#             index += 1
#         except NoSuchElementException:
#             break
#     return ids

# # 전체 후기 페이지 ID크롤링 
# def find_id():
#     z = 1
#     while True:
#         xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{z}]/div[1]/div[1]/div[1]/a/span'
#         try:
#             # id_element = driver.find_element(By.XPATH, xpath)
#             id_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath)))
#             ID.append(id_element.text)
#             z += 1
#             time.sleep(wait_time)
#         except NoSuchElementException:
#             break

# # 전체 ID, 후기, 날짜 크롤링
# def find_total():
#     g = 1
#     while True:
#         xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
#         xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
#         xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'
#         try:
#             id_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
#             ID.append(id_element.text)
#             time.sleep(wait_time)
#             date_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
#             Date.append(date_element.text)
#             time.sleep(wait_time)
#             rv_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
#             time.sleep(wait_time)
#             review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
#             time.sleep(wait_time)
#             if review_text:  # 리뷰 텍스트가 존재하면
#                 Review_text.append(review_text)
#             else:  # 리뷰 텍스트가 없는 경우
#                 Review_text.append('정보 없음')
#             g += 1
#             time.sleep(wait_time)
#             # 스크롤을 조정하여 다음 요소가 화면에 나타날 수 있도록 함
#             # 수직으로 600px만큼 아래로 스크롤
#             if g % 3 == 0:
#                 driver.execute_script("window.scrollBy(0, 600);")  
#                 time.sleep(wait_time)
#         except NoSuchElementException:
#             break

        
# 전체 후기 페이지 ID크롤링 
def find_id():
    z = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{z}]/div[1]/div[1]/div[1]/a/span'
        try:
            id_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            ID.append(id_element.text)
            z += 1
            time.sleep(wait_time)
            # 스크롤을 조정하여 다음 요소가 화면에 나타날 수 있도록 함
            # 수직으로 600px만큼 아래로 스크롤
            if z % 3 == 0:
                driver.execute_script("window.scrollBy(0, 600);")  
                time.sleep(wait_time)
        except NoSuchElementException:
            break
  
# 전체 후기 페이지 날짜 크롤링 
def find_date():
    a = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{a}]/div[1]/div[1]/div[2]/span[6]'
        try:
            # date_element = driver.find_element(By.XPATH, xpath)
            date_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            Date.append(date_element.text)
            a += 1
            time.sleep(wait_time)
            if a % 3 == 0:
                driver.execute_script("window.scrollBy(0, 600);")  
                time.sleep(wait_time)
        except NoSuchElementException:
            break
# # 전체 후기 페이지 후기 크롤링 
# def find_rv():
#     b = 1
#     while True:
#         xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{b}]/div[1]/div[3]'
#         try:
#             rv_element = driver.find_element(By.XPATH, xpath)
#             Review_text.append(rv_element.text)
#             b += 1
#             time.sleep(wait_time)
#         except NoSuchElementException:
#             break 

# 전체 후기 페이지 리뷰 크롤링 
def find_rv():
    b = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{b}]/div[1]/div[*]/p/span'
        try:
            # rv_element = driver.find_element(By.XPATH, xpath)
            # Review_text.append(rv_element.text)
            # b += 1
            # rv_element = driver.find_element(By.XPATH, xpath)
            rv_element = WebDriverWait(driver, wait_time).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
            if review_text:  # 리뷰 텍스트가 존재하면
                Review_text.append(review_text)
            else:  # 리뷰 텍스트가 없는 경우
                Review_text.append('정보 없음')
            b += 1
            time.sleep(wait_time)
            if b % 3 == 0:
                driver.execute_script("window.scrollBy(0, 600);")  
                time.sleep(wait_time)
        except NoSuchElementException:
            break
        
# 후기 더보기 누르기, 후기 접기는 정지
def class_click():
    prev_class = 'link_more'
    while True:
        element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a')
        element_class = element.get_attribute("class")    
        # 이전 클래스와 현재 클래스가 다른지 확인
        if prev_class != element_class:
            print("Class changed. Stopping click.")
            break
        # 클릭하고 잠시 대기
        element.click()
        time.sleep(wait_time)
        
        # 현재 클래스를 이전 클래스에 저장
        prev_class = element_class
store = []
ID = []
Date = []
Review_text = []

In [ ]:
# 완성 2번
# for i in range(1, len(all_results) + 1):
for i in range(9,10):
    # 배너 넘기기
    if i == 3:
        continue
    
    # 2~5초 랜덤한 대기 시간
    wait_time = random.uniform(1, 5)
    # 가게 이름 클릭
    click_def(i)
    time.sleep(wait_time)
    # 가게 이름 크롤링
    cw = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/h2').text
    # store.append(cw)
    time.sleep(wait_time)
    
    # 스크롤 내릴 위치 찾기
    try:
        more_reviews_link = driver.find_element(By.CLASS_NAME, 'rate_intro')
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 후기 더보기 링크가 보이도록 스크롤
    scroll_to_element(more_reviews_link)
    time.sleep(wait_time)
    
    # 첫번째 리뷰누르기 - 전체 리뷰 페이지로 넘어가기
    try:
        driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/ul[*]/li[1]/div[1]/div[2]/p').click()
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 전체 후기 페이지의 더보기 버튼 클릭 횟수 카운트
    click_count = 0
    prev_id_count = 0
    # 첫 번째 더보기 버튼 클릭
    class_click()
    click_count += 1
    time.sleep(wait_time)
    
    # 스크롤 위로 올리기
    # driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")

    # # ID 크롤링
    # id_list = scrape_ids()
    # id_count = len(id_list)
    # while True:
    #     # 첫 번째 더보기 버튼 클릭
    #     class_click()
    #     click_count += 1
    #     time.sleep(wait_time)
    #     # ID 크롤링
    #     id_list = scrape_ids()
    #     id_count = len(id_list)
        
    #     # 현재 ID 갯수와 이전 ID 갯수 비교
    #     if id_count <= prev_id_count:
    #         break

    #     prev_id_count = id_count
    time.sleep(wait_time)
    # for k in range(click_count - 1):
    #     load_more()
    # 스크롤 올리기
    driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")
    # find_total()
    find_id()
    #스크롤 올리기
    driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")
    find_date()
    # 스크롤 올리기
    driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")
    find_rv()
    time.sleep(wait_time)
    # 가게 이름 후기 갯수 만큼 늘리기
    # store.append(cw)
    # extended_cw = [cw for _ in range(len(ID))]
    store.append([cw for _ in range(len(ID))])

    
    # 검색 결과 페이지로 돌아가기
    driver.back()
    driver.back()
    
    # 페이지가 로드될 때까지 잠시 대기
    time.sleep(wait_time)

In [6]:
# 연습 2번
store = []
ID = []
Date = []
Review_text = []
# for i in range(1, len(all_results) + 1):
for i in range(1, 4):
    # 배너 넘기기
    if i == 3:
        continue
    
    # 2~5초 랜덤한 대기 시간
    wait_time = random.uniform(1, 5)
    # 가게 이름 클릭
    click_def(i)
    time.sleep(wait_time)
    # 가게 이름 크롤링
    cw = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/h2').text
    store.append(cw)
    time.sleep(wait_time)
    
    # 스크롤 내릴 위치 찾기
    try:
        more_reviews_link = driver.find_element(By.CLASS_NAME, 'rate_intro')
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 후기 더보기 링크가 보이도록 스크롤
    scroll_to_element(more_reviews_link)
    # time.sleep(wait_time)
    
    # 첫번째 리뷰누르기 - 전체 리뷰 페이지로 넘어가기
    try:
        driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/ul[*]/li[1]/div[1]/div[2]/p').click()
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 첫 번째 더보기 버튼 클릭
    class_click()
 
    # 스크롤 올리기
    driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")

    find_data()
    store.append([cw for _ in range(len(ID))])
    
    # 검색 결과 페이지로 돌아가기
    driver.back()
    driver.back()
    
    # 페이지가 로드될 때까지 잠시 대기
    time.sleep(wait_time)

Class changed. Stopping click.


WebDriverException: Message: unknown error: cannot deserialize the result value received from Runtime.callFunctionOn
  (Session info: chrome=123.0.6312.86)
Stacktrace:
	GetHandleVerifier [0x00007FF7FDA270C2+63090]
	(No symbol) [0x00007FF7FD992D12]
	(No symbol) [0x00007FF7FD82EC65]
	(No symbol) [0x00007FF7FD83460A]
	(No symbol) [0x00007FF7FD836159]
	(No symbol) [0x00007FF7FD836200]
	(No symbol) [0x00007FF7FD86FE1B]
	(No symbol) [0x00007FF7FD896FDA]
	(No symbol) [0x00007FF7FD86A00A]
	(No symbol) [0x00007FF7FD8971F0]
	(No symbol) [0x00007FF7FD8B3412]
	(No symbol) [0x00007FF7FD896D83]
	(No symbol) [0x00007FF7FD8683A8]
	(No symbol) [0x00007FF7FD869441]
	GetHandleVerifier [0x00007FF7FDE2262D+4238301]
	GetHandleVerifier [0x00007FF7FDE5F78D+4488509]
	GetHandleVerifier [0x00007FF7FDE57A6F+4456479]
	GetHandleVerifier [0x00007FF7FDB00606+953270]
	(No symbol) [0x00007FF7FD99E5DF]
	(No symbol) [0x00007FF7FD9992B4]
	(No symbol) [0x00007FF7FD9993EB]
	(No symbol) [0x00007FF7FD989C24]
	BaseThreadInitTh

In [ ]:
# webdriverexception 처리하기- 4월 1일 해보기.
# find_data 함수에 넣어 수정할 것
while attempts < max_attempts:
    try:
        xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
        xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
        xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'

        try:
            id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
            ID.append(id_element.text)
        except WebDriverException:  # WebDriverException 포함
            ID.append('정보 없음')

        time.sleep(wait)

        try:
            date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
            Date.append(date_element.text)
        except WebDriverException:  # WebDriverException 포함
            Date.append('정보 없음')

        time.sleep(wait)

        try:
            rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
            temp_review_text = rv_element.text.strip()
            if temp_review_text:
                Review_text.append(temp_review_text)
            else:
                Review_text.append('정보 없음')
        except WebDriverException:  # WebDriverException 포함
            Review_text.append('정보 없음')

        driver.execute_script("window.scrollBy(0, 200);")
        g += 1
        attempts = 0
    except WebDriverException:  # 이 부분은 모든 요소에 대한 크롤링 시도가 실패했을 때를 처리
        # 모든 요소에 대해 '정보 없음'을 추가
        ID.append('정보 없음')
        Date.append('정보 없음')
        Review_text.append('정보 없음')
        attempts += 1

    if attempts >= max_attempts:
        print("최대 시도 횟수에 도달하였거나 더 이상 크롤링할 요소가 없습니다.")
        break


In [7]:
print(len(store))
print(len(ID))
print(len(Date))
print(len(Review_text))

1
120
120
120


In [ ]:
Review_text

In [28]:
# store1 = [cw for _ in range(102)]
store1.append(cw)
len(store1)

103

In [29]:
prct = pd.DataFrame()
prct['Store'] = store1
prct['ID'] = ID
prct['Date'] = Date
prct['Review_text'] = Review_text
prct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Store        103 non-null    object
 1   ID           103 non-null    object
 2   Date         103 non-null    object
 3   Review_text  103 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [15]:
Jongno = pd.DataFrame()

In [16]:
Jongno['Store'] = store
Jongno['ID'] = ID
Jongno['Date'] = Date
Jongno['Review_text'] = Review_text
Jongno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Store        120 non-null    object
 1   ID           120 non-null    object
 2   Date         120 non-null    object
 3   Review_text  120 non-null    object
dtypes: object(4)
memory usage: 3.9+ KB


In [30]:
jongno = pd.DataFrame()
jongno = pd.concat([Jongno, prct], ignore_index=True)
jongno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Store        223 non-null    object
 1   ID           223 non-null    object
 2   Date         223 non-null    object
 3   Review_text  223 non-null    object
dtypes: object(4)
memory usage: 7.1+ KB


In [31]:
# 스코프 서촌점만 저장됨
jongno.to_csv('jongno.csv')

In [127]:
# 연습용
# 가게이름 클릭 함수
def click_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[1]/strong' 
        try:
            driver.find_element(By.XPATH, xpath).click()
        except NoSuchElementException:
            return None

# 스크롤하여 요소가 보이도록 만드는 함수
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

# 전체 후기 페이지 더보기 버튼 클릭 함수
def load_more():
    load_more = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a/span')
    load_more.click()
    time.sleep(3)

# 전체 후기 페이지의 ID 크롤링
def scrape_ids():
    ids = []
    index = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{index}]/div[1]/div[1]/div[1]/a/span'
        try:
            # 요소가 나타날 때까지 대기
            id_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            ids.append(id_element.text)
            index += 1
        except NoSuchElementException:
            break
    return ids

# 전체 후기 페이지 ID크롤링 
def find_id():
    index = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{index}]/div[1]/div[1]/div[1]/a/span'
        try:
            # 요소가 나타날 때까지 대기
            id_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            ID.append(id_element.text)
            index += 1
        except NoSuchElementException:
            break
# 전체 후기 페이지 날짜 크롤링 
def find_date():
    a = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{a}]/div[1]/div[1]/div[2]/span[6]'
        try:
            # 요소가 나타날 때까지 대기
            date_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            Date.append(date_element.text)
            a += 1
        except NoSuchElementException:
            break
# 전체 후기 페이지 리뷰 크롤링 
def find_rv():
    b = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{b}]/div[1]/div[*]/p/span'
        try:
            # 요소가 나타날 때까지 대기
            rv_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
            review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
            if review_text:  # 리뷰 텍스트가 존재하면
                Review_text.append(review_text)
            else:  # 리뷰 텍스트가 없는 경우
                Review_text.append('정보 없음')
            b += 1
        except NoSuchElementException:
            break 

store = []
ID = []
Date = []
Review_text = []

for i in range(1, len(all_results) + 1):
# for i in range(9, 11):
    # 배너 넘기기
    if i == 3:
        continue
    
    # 2~5초 랜덤한 대기 시간
    wait_time = random.uniform(2, 5)
    # 가게 이름 클릭
    click_def(i)
    time.sleep(wait_time)
    # 가게 이름 크롤링
    cw = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/h2').text
    time.sleep(wait_time)
    
    # 스크롤 내릴 위치 찾기
    try:
        more_reviews_link = driver.find_element(By.CLASS_NAME, 'rate_intro')
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 후기 더보기 링크가 보이도록 스크롤
    scroll_to_element(more_reviews_link)
    time.sleep(2)
    
    # 첫번째 리뷰누르기 - 전체 리뷰 페이지로 넘어가기
    try:
        driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/ul[*]/li[1]/div[1]/div[2]/p').click()
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 전체 후기 페이지의 더보기 버튼 클릭 횟수 카운트
    click_count = 0
    prev_id_count = 0
    while True:
        # 첫 번째 더보기 버튼 클릭
        load_more()
        click_count += 1

        # ID 크롤링
        id_list = scrape_ids()
        time.sleep(wait_time)
        id_count = len(id_list)

        # 현재 ID 갯수와 이전 ID 갯수 비교
        if id_count <= prev_id_count:
            break

        prev_id_count = id_count
    time.sleep(wait_time)

    
    for k in range(click_count - 1):
        load_more()

    find_id()
    find_date()
    find_rv()
    time.sleep(wait_time)
    # 가게 이름 후기 갯수 만큼 늘리기

    extended_cw = [cw for _ in range(prev_id_count)]
    store.extend(extended_cw)

    
    # 검색 결과 페이지로 돌아가기
    driver.back()
    driver.back()
    
    # 페이지가 로드될 때까지 잠시 대기
    time.sleep(wait_time)


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6C19670C2+63090]
	(No symbol) [0x00007FF6C18D2D12]
	(No symbol) [0x00007FF6C176EC65]
	(No symbol) [0x00007FF6C17B499D]
	(No symbol) [0x00007FF6C17B4ADC]
	(No symbol) [0x00007FF6C17F5B37]
	(No symbol) [0x00007FF6C17D701F]
	(No symbol) [0x00007FF6C17F3412]
	(No symbol) [0x00007FF6C17D6D83]
	(No symbol) [0x00007FF6C17A83A8]
	(No symbol) [0x00007FF6C17A9441]
	GetHandleVerifier [0x00007FF6C1D6262D+4238301]
	GetHandleVerifier [0x00007FF6C1D9F78D+4488509]
	GetHandleVerifier [0x00007FF6C1D97A6F+4456479]
	GetHandleVerifier [0x00007FF6C1A40606+953270]
	(No symbol) [0x00007FF6C18DE5DF]
	(No symbol) [0x00007FF6C18D92B4]
	(No symbol) [0x00007FF6C18D93EB]
	(No symbol) [0x00007FF6C18C9C24]
	BaseThreadInitThunk [0x00007FFB1BFF257D+29]
	RtlUserThreadStart [0x00007FFB1CF8AA58+40]


In [ ]:
store

In [ ]:
Review_text

### 전체 후기 페이지의 더보기 버튼 정지시키기

In [223]:
# 더보기 버튼의 class 확인
element = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[22]/div/div[2]/a')
element_class = element.get_attribute("class")
print(element_class)


link_more link_unfold


link_more
//*[@id="mArticle"]/div[22]/div/div[2]/a
link_more link_unfold
//*[@id="mArticle"]/div[22]/div/div[2]/a
//*[@id="mArticle"]/div[18]/div/div[2]/a

In [229]:
# 후기 더보기 누르기, 후기 접기는 정지
def class_click():
    prev_class = 'link_more'
    while True:
        element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a')
        element_class = element.get_attribute("class")
        
        # 이전 클래스와 현재 클래스가 다른지 확인
        if prev_class != element_class:
            print("Class changed. Stopping click.")
            break
        
        # 클릭하고 잠시 대기
        element.click()
        time.sleep(2)
        
        # 현재 클래스를 이전 클래스에 저장
        prev_class = element_class

# 함수 호출
class_click()


Class changed. Stopping click.


In [214]:
# class 로 멈추는 함수 
# 이거 계속 누름
def class_click_2():
    while True:
        try:
            element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a/span')
            element.click()
            time.sleep(2)  # 클릭 후 잠시 대기
            
            # 클래스 이름이 "link_more link_unfold"인지 확인
            if "link_more" in element.get_attribute("class") and "link_unfold" in element.get_attribute("class"):
                break
            else:
                continue
        except StaleElementReferenceException:
            break  # 요소가 변경되었을 때 다시 찾기

class_click_2()

In [209]:
def class_click():
    while True:
        element = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[*]/div/div[2]/a/span')
        element_class = element.get_attribute("class")
        element.click()
        if "link_more" in element_class and "link_unfold" in element_class:
            
            print('stop')
        else:
            element.click()
            time.sleep(2) 
                  
class_click()

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=123.0.6312.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C19670C2+63090]
	(No symbol) [0x00007FF6C18D2D12]
	(No symbol) [0x00007FF6C176EC65]
	(No symbol) [0x00007FF6C1773D21]
	(No symbol) [0x00007FF6C1776159]
	(No symbol) [0x00007FF6C1776200]
	(No symbol) [0x00007FF6C17B6DD5]
	(No symbol) [0x00007FF6C17B5DE0]
	(No symbol) [0x00007FF6C17AA708]
	(No symbol) [0x00007FF6C17D6FDA]
	(No symbol) [0x00007FF6C17AA00A]
	(No symbol) [0x00007FF6C17D71F0]
	(No symbol) [0x00007FF6C17F3412]
	(No symbol) [0x00007FF6C17D6D83]
	(No symbol) [0x00007FF6C17A83A8]
	(No symbol) [0x00007FF6C17A9441]
	GetHandleVerifier [0x00007FF6C1D6262D+4238301]
	GetHandleVerifier [0x00007FF6C1D9F78D+4488509]
	GetHandleVerifier [0x00007FF6C1

### 전체 후기 페이지에서 크롤링하기

In [ ]:
# 전체 후기 페이지 더보기 버튼 클릭 함수
def load_more():
    load_more = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a/span')
    load_more.click()
    time.sleep(3)
# 전체 후기 페이지 ID 크롤링    
def scrape_ids():
    index = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{index}]/div[1]/div[1]/div[1]/a/span'
        try:
            id_element = driver.find_element(By.XPATH, xpath)
            ids.append(id_element.text)
            index += 1
            load_more()
        except NoSuchElementException:
            break   
ids = []       
load_more()
scrape_ids()


//*[@id="mArticle"]/div[18]/div/div[2]/ul/li[1]/div[1]/div[1]/div[1]/a/span

In [16]:
ids

['쩝문가',
 '박현아',
 '별점조작은말없이1점',
 '고영',
 'HY',
 '마제소바굳',
 '크리스토퍼',
 '.',
 '승야아찌',
 '제 평가는요.',
 '마샬리스',
 'Shinjee',
 'MK',
 '눈누난나']

In [117]:
# 완성본
# 전체 후기 페이지 더보기 버튼 클릭 함수
def load_more():
    load_more = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a/span')
    load_more.click()
    time.sleep(3)

# 전체 후기 페이지의 ID 크롤링
def scrape_ids():
    ids = []
    index = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{index}]/div[1]/div[1]/div[1]/a/span'
        try:
            id_element = driver.find_element(By.XPATH, xpath)
            ids.append(id_element.text)
            index += 1
        except NoSuchElementException:
            break
    return ids


# 전체 후기 페이지의 더보기 버튼 클릭 횟수 카운트
click_count = 0
prev_id_count = 0

while True:
    # 첫 번째 더보기 버튼 클릭
    load_more()
    click_count += 1

    # ID 크롤링
    id_list = scrape_ids()
    time.sleep(wait_time)
    id_count = len(id_list)

    # 현재 ID 갯수와 이전 ID 갯수 비교
    if id_count <= prev_id_count:
        print("더 이상 ID가 추가되지 않았습니다. 크롤링을 중단합니다.")
        break

    prev_id_count = id_count

# 전체 ID 갯수 출력
print("전체 ID 갯수:", len(id_list))
print("더보기 버튼 클릭 횟수:", click_count)
# 어제거 원본 / 3시 37분 실행

더 이상 ID가 추가되지 않았습니다. 크롤링을 중단합니다.
전체 ID 갯수: 10
더보기 버튼 클릭 횟수: 5


In [97]:
prev_id_count

42

In [98]:
id_count

10

1. 가게 이름 누르고 들어간다
2. 가게 이름 크롤링
3. rate_intro 찾아서 있으면, 첫번째 후기 클릭해서 전체 후기 들어가고 뒤로가기 2번
   없으면, 뒤로가기 1번

In [120]:
# 완성본
# 더보기 버튼 클릭 함수
def load_more():
    load_more = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[*]/div/div[2]/a/span')
    load_more.click()
    time.sleep(3)

# 전체 후기 페이지 ID크롤링 
def find_id():
    index = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{index}]/div[1]/div[1]/div[1]/a/span'
        try:
            id_element = driver.find_element(By.XPATH, xpath)
            ID.append(id_element.text)
            index += 1
        except NoSuchElementException:
            break
        
# 전체 후기 페이지 날짜 크롤링 
def find_date():
    a = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{a}]/div[1]/div[1]/div[2]/span[6]'
        try:
            date_element = driver.find_element(By.XPATH, xpath)
            Date.append(date_element.text)
            a += 1
        except NoSuchElementException:
            break
        
# 전체 후기 페이지 리뷰 크롤링 
def find_rv():
    b = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{b}]/div[1]/div[*]/p/span'
        try:
            # rv_element = driver.find_element(By.XPATH, xpath)
            # Review_text.append(rv_element.text)
            # b += 1
            rv_element = driver.find_element(By.XPATH, xpath)
            review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
            if review_text:  # 리뷰 텍스트가 존재하면
                Review_text.append(review_text)
            else:  # 리뷰 텍스트가 없는 경우
                Review_text.append('정보 없음')
            b += 1
        except NoSuchElementException:
            break
 
# 전체 후기 페이지 평점 크롤링 
def find_point():
    e = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[18]/div/div[2]/ul/li[{e}]/div[1]/div[1]/div[3]/span/span'
        try:
            pt_element = driver.find_element(By.XPATH, xpath)
            Point.append(pt_element.text)
            e += 1
        except NoSuchElementException:
            break       
ID = []
Date = []
Review_text = []
Point = []
for k in range(click_count - 1):
    load_more()

# find_id()
# find_date()
# find_rv()
find_point()

In [121]:
Point

[]

//*[@id="mArticle"]/div[18]/div/div[2]/ul/li[19]/div[1]/div[1]/div[3]/span/span
//*[@id="mArticle"]/div[18]/div/div[2]/ul/li[18]/div[1]/div[1]/div[3]/span/span
//*[@id="mArticle"]/div[18]/div/div[2]/ul/li[4]/div[1]/div[1]/div[3]/span/span
//*[@id="mArticle"]/div[18]/div/div[2]/ul/li[5]/div[1]/div[1]/div[3]/span/span

//*[@id="mArticle"]/div[20]/div/div[2]/ul/li[57]/div[1]/div[2]/p/span
//*[@id="mArticle"]/div[20]/div/div[2]/ul/li[56]/div[1]/div[2]/p/span
//*[@id="mArticle"]/div[20]/div/div[2]/ul/li[52]/div[1]/div[2]/p/span
//*[@id="mArticle"]/div[20]/div/div[2]/ul/li[47]/div[1]/div[3]/p/span
//*[@id="mArticle"]/div[18]/div/div[2]/ul/li[2]/div[1]/div[2]/p/span
//*[@id="mArticle"]/div[18]/div/div[2]/ul/li[4]/div[1]/div[3]/p/span

In [89]:
len(Review_text)

41

In [90]:
Review_text

['정보 없음',
 '맛은 있는데 안내가 너무 부족합니다.',
 '정보 없음',
 '기존의 소금빵과 다르게 좀 기름진 느낌\n근데 맛있는 기름짐.. 커피랑 마시면 무한 흡입 가능..',
 '버터 풍미가 엄청! 강합니다. 그리고 버터의 느끼함을 잡으려고 그런건지 소금맛도 강하게 느껴집니다. 무엇보다 파샥 촉촉 식감이 진짜 훌륭해요. 다만 저는 담백한 소금빵파라 저한텐 좀 느끼했네요…',
 '정보 없음',
 '정보 없음',
 '바삭하고 부드럽고 버터가득 존맛',
 '기름지고 느끼한 소금빵 원ㄴ하는 사람 제발 사세요\n난 기름진거 좋아해서 너무너무 좋앗음\n버터향 대박',
 '따뜻할 때 바로 먹었는데 바삭하고 버터풍미가 확 올라와서 좋았어요. 포장이 느린 게 조금 아쉬웠지만 맛있어서 재방문의사 있음',
 '포장해서 갔지만, 직원들 친절하고, 분위기도 괜찮은듯함 ~~',
 '조금 비싼 느낌이 있지만 맛있긴 함 또 갈 듯',
 'JMT',
 '결제 먼저 하고 줄 서야한다는것에대한 안내가 부족합니다. 안내판보다 한참 뒤까지 사람들이 줄을 서있어서 한참 대기 후 결제하고 돌아와 다시 줄을 서서 대기시간 두배로 길어지는 사람들이 30프로정도는 되더군요. 직원이 가끔씩 나와서 안내를 하던, 안내판을 뒤쪽 줄 선 사람들도 볼수있도록 위치해두시던지 해야할듯.\n\n식음료 매장 재방문률 높이는 원칙 몇가지 중 하나가 고객에게 불쾌한 경험 주지 않기던데.. 그 부분에서는 실패네요. 배달전문점이 아니니 그부분은 개선하셔야겠네요.\n\n맛은 무난했습니다. 서울에서 이정도 가격 소금빵은 저렴한편이라 가성비는 나쁘지않네요. 하지만 익선동은 가성비 따지려고 오는곳이 아닌지라..',
 '소금빵안에도 버터가 들어가는데 밖에도 버터를 넘 많아 발라 느끼하네요ㅜㅜ',
 '1세트 4개!\n12000원 \n소금빵 진짜 진짜 맛있네요ㅎㅎ',
 '너무 맛있네요',
 '정보 없음',
 '맛은 있지만 직원이 좀 짜증이 많은거같음..\n포장은 정성스럽지만, 포장 때문에 받는데 시간이 너무 오래걸려서 다른 방식으

In [58]:
len(ID)

20

In [59]:
len(Date)

20

In [11]:
# 더보기 버튼 클릭 함수
def load_more():
    load_more = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[*]/div/div[2]/a/span')
    load_more.click()
    time.sleep(3)

# 전체 ID 크롤링
def scrape_ids():
    ids = []
    index = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{index}]/div[1]/div[1]/div[1]/a/span'
        try:
            id_element = driver.find_element(By.XPATH, xpath)
            ids.append(id_element.text)
            index += 1
        except NoSuchElementException:
            break
    return ids

# 더보기 버튼 클릭 횟수 카운트
click_count = 0
prev_id_count = 0

while True:
    # 첫 번째 더보기 버튼 클릭
    load_more()
    click_count += 1

    # ID 크롤링
    id_list = scrape_ids()
    id_count = len(id_list)

    # 현재 ID 갯수와 이전 ID 갯수 비교
    if id_count <= prev_id_count:
        print("더 이상 ID가 추가되지 않았습니다. 크롤링을 중단합니다.")
        break

    prev_id_count = id_count

# 전체 ID 갯수 출력
print("전체 ID 갯수:", len(id_list))
print("더보기 버튼 클릭 횟수:", click_count)

더 이상 ID가 추가되지 않았습니다. 크롤링을 중단합니다.
전체 ID 갯수: 0
더보기 버튼 클릭 횟수: 1


In [145]:
# 전체 후기 페이지
# 더보기 버튼 클릭 함수
def load_more():
    load_more = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a/span')
    load_more.click()
    time.sleep(3)

# 전체 ID 크롤링
def scrape_ids():
    ids = []
    index = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[16]/div/div[2]/ul/li[{index}]/div[1]/div[1]/div[1]/a/span'
        try:
            id_element = driver.find_element(By.XPATH, xpath)
            ids.append(id_element.text)
            index += 1
        except NoSuchElementException:
            break
    return ids

# 더보기 버튼 클릭 횟수 카운트
click_count = 0
prev_id_count = 0

while True:
    # 첫 번째 더보기 버튼 클릭
    load_more()
    click_count += 1

    # ID 크롤링
    id_list = scrape_ids()
    id_count = len(id_list)

    # 현재 ID 갯수와 이전 ID 갯수 비교
    if id_count <= prev_id_count:
        print("더 이상 ID가 추가되지 않았습니다. 크롤링을 중단합니다.")
        break

    prev_id_count = id_count

# 전체 ID 갯수 출력
print("전체 ID 갯수:", len(id_list))
print("더보기 버튼 클릭 횟수:", click_count)

더 이상 ID가 추가되지 않았습니다. 크롤링을 중단합니다.
전체 ID 갯수: 0
더보기 버튼 클릭 횟수: 1


In [84]:
# 더보기 버튼 클릭 함수
def load_more():
    try:
        lomore = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[*]/div/div[2]/a/span')
        lomore.click()
        time.sleep(3)
    except NoSuchElementException:
        pass

# 전체 ID 크롤링
def scrape_ids():
    # ids = []
    index = 1
    while True:
        xpath = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{index}]/div[1]/div[1]/div[1]/a/span'
        try:
            id_element = driver.find_element(By.XPATH, xpath)
            # ids.append(id_element.text)
            ID.append(id_element.text)
            index += 1
        except NoSuchElementException:
            break
        
ID = []
while True:
    # 첫 번째 더보기 버튼 클릭
    load_more()
    click_count += 1

    # ID 크롤링
    id_list = scrape_ids()
    ID.append(id_list)
    
    # 현재 ID 갯수와 이전 ID 갯수 비교
    if id_count <= prev_id_count:
        break

    prev_id_count = id_count

In [86]:
len(ID)

21